# EV charging Data Analysis

Site name - Urban Science Building (USB), Newcastle University

Import necessary libraries:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Data Collection:

The data is retreived from Fastned and Newcastle University.


In [2]:
sessions_data = pd.read_csv(r"D:\Study\My Projects\Analysis of EV charging usage in UK\data\session.csv")

In [3]:
sessions_data.head()

,id,sessionStart,sessionStop,connectorType,kwh,authType,authId,chargerId,locationId
0,1ff7bb21-86f6-4a58-9713-63c9551b0dc3,2021-03-18 17:39:36,2021-03-18 17:59:06,IEC_62196_T2_COMBO,9509,CUSTOMER,1062232,5000198,50112
1,4ec4bf07-f359-4199-93d3-ec842a450cfa,2021-03-18 19:00:16,2021-03-18 19:33:43,CHADEMO,6566,CUSTOMER,1056758,5000194,50112
2,63ef2126-e813-4225-982d-fcd89108f2ba,2021-03-18 20:43:25,2021-03-18 21:07:50,CHADEMO,13882,CUSTOMER,1076569,5000197,50112
3,707dbb44-cb45-4cea-ab86-dec7c06d6a43,2021-03-18 21:23:07,2021-03-18 21:47:35,IEC_62196_T2_COMBO,18428,CUSTOMER,1073736,5000198,50112
4,55a243f9-6c3a-4a53-a964-fc7f3d88ddb7,2021-03-18 22:24:24,2021-03-18 23:09:28,CHADEMO,12836,CUSTOMER,1073736,5000197,50112


The data represents the record of charging sessions over a period of 4 years from March 2021 to June 2024.
The initial rows of the data are shown above. The columns of this data represent:

- _id_ - Id of each session
- _sessionStart_ - The date and time of the start of a charging session.
- _sessionStop_ - The date and time of the end of a charging session.
- _connectorType_ - The type of connector the car was connected during the session
- _kwh_ - The energy delivered during each session.
- _authType_ - The mode of authentication/payment used for charging.
- _authId_ - The authentication Id.
- _chargerId_ - The Id of the Charger.
- _locationId_ - The Id of the location of the charger which is the same for all.

Among these columns, Location Id is irrelavant to our analysis as it is same for all. Hence, those columns will be dropped from the data.

## 2. Data Preparation:

In [6]:
usb_data = sessions_data.drop(columns={'locationId'})

In [7]:
usb_data.head()

,id,sessionStart,sessionStop,connectorType,kwh,authType,authId,chargerId
0,1ff7bb21-86f6-4a58-9713-63c9551b0dc3,2021-03-18 17:39:36,2021-03-18 17:59:06,IEC_62196_T2_COMBO,9509,CUSTOMER,1062232,5000198
1,4ec4bf07-f359-4199-93d3-ec842a450cfa,2021-03-18 19:00:16,2021-03-18 19:33:43,CHADEMO,6566,CUSTOMER,1056758,5000194
2,63ef2126-e813-4225-982d-fcd89108f2ba,2021-03-18 20:43:25,2021-03-18 21:07:50,CHADEMO,13882,CUSTOMER,1076569,5000197
3,707dbb44-cb45-4cea-ab86-dec7c06d6a43,2021-03-18 21:23:07,2021-03-18 21:47:35,IEC_62196_T2_COMBO,18428,CUSTOMER,1073736,5000198
4,55a243f9-6c3a-4a53-a964-fc7f3d88ddb7,2021-03-18 22:24:24,2021-03-18 23:09:28,CHADEMO,12836,CUSTOMER,1073736,5000197


Now, the kwh column will be renamed for better understanding and divided by 1000 to make it actually kWh which was just Wh (verified with the data owner.)

In [8]:
usb_data.rename(columns={'kwh' : 'energy_consumed_kWh'}, inplace=True)

In [9]:
usb_data['energy_consumed_kWh'] = usb_data['energy_consumed_kWh'] / 1000

In [10]:
usb_data.head()

,id,sessionStart,sessionStop,connectorType,energy_consumed_kWh,authType,authId,chargerId
0,1ff7bb21-86f6-4a58-9713-63c9551b0dc3,2021-03-18 17:39:36,2021-03-18 17:59:06,IEC_62196_T2_COMBO,9.509,CUSTOMER,1062232,5000198
1,4ec4bf07-f359-4199-93d3-ec842a450cfa,2021-03-18 19:00:16,2021-03-18 19:33:43,CHADEMO,6.566,CUSTOMER,1056758,5000194
2,63ef2126-e813-4225-982d-fcd89108f2ba,2021-03-18 20:43:25,2021-03-18 21:07:50,CHADEMO,13.882,CUSTOMER,1076569,5000197
3,707dbb44-cb45-4cea-ab86-dec7c06d6a43,2021-03-18 21:23:07,2021-03-18 21:47:35,IEC_62196_T2_COMBO,18.428,CUSTOMER,1073736,5000198
4,55a243f9-6c3a-4a53-a964-fc7f3d88ddb7,2021-03-18 22:24:24,2021-03-18 23:09:28,CHADEMO,12.836,CUSTOMER,1073736,5000197


Now the columns will be reordered to place all types of IDs at the start along with the authType to make the data more sensible.

In [11]:
usb_data = usb_data.iloc[:, [0, 6, 7, 5, 1, 2, 3, 4]]

In [12]:
usb_data.head()

,id,authId,chargerId,authType,sessionStart,sessionStop,connectorType,energy_consumed_kWh
0,1ff7bb21-86f6-4a58-9713-63c9551b0dc3,1062232,5000198,CUSTOMER,2021-03-18 17:39:36,2021-03-18 17:59:06,IEC_62196_T2_COMBO,9.509
1,4ec4bf07-f359-4199-93d3-ec842a450cfa,1056758,5000194,CUSTOMER,2021-03-18 19:00:16,2021-03-18 19:33:43,CHADEMO,6.566
2,63ef2126-e813-4225-982d-fcd89108f2ba,1076569,5000197,CUSTOMER,2021-03-18 20:43:25,2021-03-18 21:07:50,CHADEMO,13.882
3,707dbb44-cb45-4cea-ab86-dec7c06d6a43,1073736,5000198,CUSTOMER,2021-03-18 21:23:07,2021-03-18 21:47:35,IEC_62196_T2_COMBO,18.428
4,55a243f9-6c3a-4a53-a964-fc7f3d88ddb7,1073736,5000197,CUSTOMER,2021-03-18 22:24:24,2021-03-18 23:09:28,CHADEMO,12.836


In [21]:
usb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34432 entries, 0 to 34431
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   34432 non-null  object 
 1   authId               34432 non-null  int64  
 2   chargerId            34432 non-null  int64  
 3   authType             34432 non-null  object 
 4   sessionStart         34432 non-null  object 
 5   sessionStop          34432 non-null  object 
 6   connectorType        34432 non-null  object 
 7   energy_consumed_kWh  34432 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 2.1+ MB


Now let's convert the date and time from general object to date-time object to ensure it is in right format.

In [22]:
usb_data['sessionStart'] = pd.to_datetime(usb_data['sessionStart'])
usb_data['sessionStop'] = pd.to_datetime(usb_data['sessionStop'])

In [23]:
usb_data.head()

,id,authId,chargerId,authType,sessionStart,sessionStop,connectorType,energy_consumed_kWh
0,1ff7bb21-86f6-4a58-9713-63c9551b0dc3,1062232,5000198,CUSTOMER,2021-03-18 17:39:36,2021-03-18 17:59:06,IEC_62196_T2_COMBO,9.509
1,4ec4bf07-f359-4199-93d3-ec842a450cfa,1056758,5000194,CUSTOMER,2021-03-18 19:00:16,2021-03-18 19:33:43,CHADEMO,6.566
2,63ef2126-e813-4225-982d-fcd89108f2ba,1076569,5000197,CUSTOMER,2021-03-18 20:43:25,2021-03-18 21:07:50,CHADEMO,13.882
3,707dbb44-cb45-4cea-ab86-dec7c06d6a43,1073736,5000198,CUSTOMER,2021-03-18 21:23:07,2021-03-18 21:47:35,IEC_62196_T2_COMBO,18.428
4,55a243f9-6c3a-4a53-a964-fc7f3d88ddb7,1073736,5000197,CUSTOMER,2021-03-18 22:24:24,2021-03-18 23:09:28,CHADEMO,12.836


### 2.1 Data Cleaning:

As the data is perfectly ordered and in right format, now it will undergo some cleaning which will include removing of null or invalid values of kWh. 

In [13]:
usb_data.isnull().sum()

id                     0
authId                 0
chargerId              0
authType               0
sessionStart           0
sessionStop            0
connectorType          0
energy_consumed_kWh    0
dtype: int64

There are no null values. Now lets check the central tendencies to identify the range of data

In [14]:
usb_data.describe()

,authId,chargerId,energy_consumed_kWh
count,3.443200e+04,3.443200e+04,34432.000000
mean,9.813461e+06,5.000197e+06,26.604024
std,1.986829e+07,1.762099e+00,19.522729
min,1.130000e+02,5.000194e+06,0.000000
25%,1.110206e+06,5.000195e+06,10.188750
50%,1.211702e+06,5.000197e+06,23.467000
75%,4.178835e+06,5.000198e+06,40.150000
max,1.572480e+08,5.000199e+06,110.733000


As it can be seen, the least energy consumed is zero. It is significant to identify how many zero values are there in the data.

In [20]:
usb_data[usb_data['energy_consumed_kWh'] == 0]['energy_consumed_kWh'].count()

577

Now the new data will have sessions which have consumed energy more than 5 kWh.

In [32]:
usb_data_filtered = usb_data[usb_data['energy_consumed_kWh'] > 5]

In [33]:
usb_data_filtered.describe()

,authId,chargerId,sessionStart,sessionStop,energy_consumed_kWh
count,2.977400e+04,2.977400e+04,29774,29774,29774.000000
mean,1.032042e+07,5.000197e+06,2023-02-21 11:12:57.897427200,2023-02-21 11:57:27.335460608,30.441438
min,1.130000e+02,5.000194e+06,2021-03-18 17:39:36,2021-03-18 17:59:06,5.002000
25%,1.110689e+06,5.000195e+06,2022-06-27 05:31:01,2022-06-27 06:12:22,14.908500
50%,1.208763e+06,5.000197e+06,2023-03-22 10:27:46,2023-03-22 11:28:56.500000,27.405500
75%,6.469794e+06,5.000199e+06,2023-11-23 15:31:31.750000128,2023-11-23 15:55:35.249999872,42.911500
max,1.572480e+08,5.000199e+06,2024-07-01 15:20:10,2024-07-01 15:52:45,110.733000
std,2.037021e+07,1.745373e+00,NaN,NaN,18.207093


Now let's calculate the duration of each charging session:

In [51]:
usb_data_filtered['duration(hrs)'] = ((usb_data_filtered['sessionStop'] - usb_data_filtered['sessionStart']).dt.total_seconds()) / 3600
usb_data_filtered['duration(mins)'] = ((usb_data_filtered['sessionStop'] - usb_data_filtered['sessionStart']).dt.total_seconds()) / 60

C:\Users\Kaustubh\AppData\Local\Temp\ipykernel_12448\619619653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usb_data_filtered['duration(hrs)'] = ((usb_data_filtered['sessionStop'] - usb_data_filtered['sessionStart']).dt.total_seconds()) / 3600
C:\Users\Kaustubh\AppData\Local\Temp\ipykernel_12448\619619653.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usb_data_filtered['duration(mins)'] = ((usb_data_filtered['sessionStop'] - usb_data_filtered['sessionStart']).dt.total_seconds()) / 60


In [52]:
usb_data_filtered.head()

,id,authId,chargerId,authType,sessionStart,sessionStop,connectorType,energy_consumed_kWh,duration(hrs),duration(mins)
0,1ff7bb21-86f6-4a58-9713-63c9551b0dc3,1062232,5000198,CUSTOMER,2021-03-18 17:39:36,2021-03-18 17:59:06,IEC_62196_T2_COMBO,9.509,0.325000,19.500000
1,4ec4bf07-f359-4199-93d3-ec842a450cfa,1056758,5000194,CUSTOMER,2021-03-18 19:00:16,2021-03-18 19:33:43,CHADEMO,6.566,0.557500,33.450000
2,63ef2126-e813-4225-982d-fcd89108f2ba,1076569,5000197,CUSTOMER,2021-03-18 20:43:25,2021-03-18 21:07:50,CHADEMO,13.882,0.406944,24.416667
3,707dbb44-cb45-4cea-ab86-dec7c06d6a43,1073736,5000198,CUSTOMER,2021-03-18 21:23:07,2021-03-18 21:47:35,IEC_62196_T2_COMBO,18.428,0.407778,24.466667
4,55a243f9-6c3a-4a53-a964-fc7f3d88ddb7,1073736,5000197,CUSTOMER,2021-03-18 22:24:24,2021-03-18 23:09:28,CHADEMO,12.836,0.751111,45.066667


Now we have the duration in hours and in minutes as well. Let's check the statistics.

In [53]:
usb_data_filtered.describe()

,authId,chargerId,sessionStart,sessionStop,energy_consumed_kWh,duration(hrs),duration(mins)
count,2.977400e+04,2.977400e+04,29774,29774,29774.000000,29774.000000,29774.000000
mean,1.032042e+07,5.000197e+06,2023-02-21 11:12:57.897427200,2023-02-21 11:57:27.335460608,30.441438,0.741511,44.490634
min,1.130000e+02,5.000194e+06,2021-03-18 17:39:36,2021-03-18 17:59:06,5.002000,0.035833,2.150000
25%,1.110689e+06,5.000195e+06,2022-06-27 05:31:01,2022-06-27 06:12:22,14.908500,0.416667,25.000000
50%,1.208763e+06,5.000197e+06,2023-03-22 10:27:46,2023-03-22 11:28:56.500000,27.405500,0.644167,38.650000
75%,6.469794e+06,5.000199e+06,2023-11-23 15:31:31.750000128,2023-11-23 15:55:35.249999872,42.911500,0.950000,57.000000
max,1.572480e+08,5.000199e+06,2024-07-01 15:20:10,2024-07-01 15:52:45,110.733000,12.570278,754.216667
std,2.037021e+07,1.745373e+00,NaN,NaN,18.207093,0.486146,29.168738
